In [1]:
import pyspark
from pyspark.sql import SparkSession, types

In [1]:
!wget https://datasets.imdbws.com/title.episode.tsv.gz

--2023-04-09 16:45:16--  https://datasets.imdbws.com/title.episode.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 18.65.229.63, 18.65.229.92, 18.65.229.126, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|18.65.229.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40432108 (39M) [binary/octet-stream]
Saving to: ‘title.episode.tsv.gz’

title.episode.tsv.g 100%[===================>]  38.56M  57.8MB/s    in 0.7s    

2023-04-09 16:45:17 (57.8 MB/s) - ‘title.episode.tsv.gz’ saved [40432108/40432108]



In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 16:01:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/23 16:01:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('title.episode.tsv.gz', sep='\t')

In [7]:
df.count()

7407960

In [8]:
df.show()

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|          \N|           \N|
|tt0043426|   tt0040051|           3|           42|
|tt0043631|   tt0989125|           2|           16|
|tt0043693|   tt0989125|           2|            8|
|tt0043710|   tt0989125|           3|            3|
|tt0044093|   tt0959862|           1|            6|
|tt0044668|   tt0044243|           2|           16|
|tt0044901|   tt0989125|           3|           46|
|tt0045519|   tt0989125|           4|           11|
|tt0045960|   tt0044284|           2|            3|
|tt0046135|   tt0989125|           4|            5|
|tt0046150|   tt0341798|          \N|           \N|
|tt0046855|   tt0046643|           1|            4|
|tt0046864|   tt0989125|           5|           20|
|tt0047810| 

In [9]:
from pyspark.sql import types
title_schema = types.StructType([
    types.StructField('tconst', types.StringType(), True),
    types.StructField('parentTconst', types.StringType(), True),
    types.StructField('seasonNumber', types.IntegerType(), True),
    types.StructField('episodeNumber', types.IntegerType(), True),
])

In [ ]:
'''
schema = {
    tconst = Column(String), # ID
    parentTconst = Column(String), # FK: title_basics?
    seasonNumber = Column(Integer),
    episodeNumber = Column(Integer)
}
'''

In [11]:
df = spark.read \
    .option("header", "true") \
    .csv(
        'title.episode.tsv.gz', 
        sep='\t',
        schema=title_schema
    )

In [12]:
df.loc[df['seasonNumber'] == '\\N', 'seasonNumber'] = np.nan
df.loc[df['episodeNumber'] == '\\N', 'episodeNumber'] = np.nan

NameError: name 'np' is not defined

In [27]:
# df['seasonNumber'] = df['seasonNumber'].astype('int')
# df['episodeNumber'] = df['episodeNumber'].astype('int')
df['seasonNumber'].map(type) == str

0           True
1           True
2          False
3           True
4           True
           ...  
7407955     True
7407956     True
7407957     True
7407958     True
7407959     True
Name: seasonNumber, Length: 7407960, dtype: bool